In [260]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader
from thermography_dataset_one_layer import ThermDataset
from sklearn.model_selection import train_test_split
%matplotlib inline

In [261]:
args = {'lr':0.01,
        'epochs':1000,
        'noise':0.001,
        'train size':0.7,
        'spec scale':10**12
        }

In [262]:
def df_to_tensor(df):
    return torch.tensor(df.values, dtype=torch.float32)

# Prepare Data

In [263]:
df = pd.read_excel('combined_data.xlsx')

In [264]:
X = df.iloc[:,11:]
y = df.iloc[:,:11]

In [265]:
X = X.apply(lambda x: x*args['spec scale'])

In [266]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=args['train size'], random_state=101)

In [267]:
X_train

,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
805,0.532133,0.638418,0.726998,0.802254,0.866701,0.933640,1.000501,1.067615,1.134905,1.202684,...,5.303224,5.372908,5.430458,5.471816,5.494759,5.486708,5.423339,5.281082,5.076630,4.888354
1798,0.414041,0.457451,0.496973,0.537416,0.575507,0.616503,0.658605,0.701921,0.746277,0.791765,...,4.100028,4.163747,4.216513,4.255809,4.280443,4.280814,4.237858,4.132937,3.978863,3.836942
193,0.465971,0.535950,0.599500,0.657984,0.709998,0.764922,0.820523,0.876953,0.934031,0.991913,...,4.857763,4.927424,4.984310,5.025386,5.049202,5.044452,4.988772,4.860395,4.674584,4.503457
162,0.383691,0.450965,0.514631,0.572481,0.623304,0.676948,0.731329,0.786481,0.842163,0.898461,...,4.744066,4.813846,4.870592,4.911579,4.935582,4.931638,4.877883,4.753013,4.571928,4.405149
1211,0.202562,0.201751,0.203200,0.210987,0.221130,0.232812,0.245369,0.258871,0.273278,0.288644,...,1.673267,1.706698,1.737154,1.762956,1.783029,1.793023,1.784698,1.749866,1.693567,1.641708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,0.289292,0.303189,0.315865,0.332897,0.350998,0.370787,0.391281,0.412664,0.434948,0.458279,...,2.185162,2.223102,2.258088,2.287536,2.309717,2.318868,2.304381,2.255814,2.179809,2.109786
1599,0.381550,0.470170,0.548909,0.615927,0.673070,0.732666,0.792570,0.852905,0.913487,0.974471,...,5.003251,5.074765,5.132710,5.174146,5.197733,5.191909,5.133690,5.000708,4.808701,4.631874
1361,0.453996,0.544847,0.620312,0.684716,0.740257,0.798081,0.856012,0.914347,0.973024,1.032310,...,4.783330,4.849700,4.904975,4.945541,4.969423,4.965237,4.910910,4.784997,4.602506,4.434433
1547,0.325858,0.330976,0.332963,0.341853,0.353758,0.367173,0.381218,0.396176,0.412134,0.429284,...,1.653589,1.682302,1.710493,1.735484,1.755333,1.765342,1.757317,1.723186,1.667904,1.616983


# Design Models

In [268]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(input_size, 45)
        self.lin2 = nn.Linear(45, 60)
        self.lin3 = nn.Linear(60, 75)
        self.lin4 = nn.Linear(75, 60)
        self.lin_fin = nn.Linear(60, output_size)


    def forward(self, x):
        x = F.leaky_relu(self.lin1(x))
        x = F.leaky_relu(self.lin2(x))
        x = F.leaky_relu(self.lin3(x))
        x = F.leaky_relu(self.lin4(x))
        x = self.lin_fin(x)
        return x

In [269]:
n_samples = len(X_train)
input_size = 66
output_size = 1

# store models in descending order (11, 10, 9...)
models = []
for i in range(11):
    models.append(Net(input_size+i, output_size))

# Training Loops

In [270]:
learning_rate = args['lr']
criterion = nn.MSELoss()

In [271]:
for i in range(11):
    best_loss = np.inf
    best_epoch = 0

    print(f'Layer {11-i}')

    optimizer = torch.optim.Adam(models[i].parameters(), lr=learning_rate)
    model = models[i]

    inputs = df_to_tensor(X_train)
    outputs = df_to_tensor(y_train.iloc[:,10-i]).reshape(-1,1)

    for epoch in range(args['epochs']):
        # empty gradients
        optimizer.zero_grad()

        # forward pass
        pred = model(inputs)

        # loss
        loss = criterion(pred, outputs)

        # backward pass
        loss.backward()
        
        # update
        optimizer.step()

        #if (epoch+1) % int(args['epochs']/10) == 0:
        #    print(f'epoch: {epoch+1}, loss = {loss}')

        if loss < best_loss:
            best_loss = loss
            best_epoch = epoch + 1

    pred_np = pred.detach().numpy()
    #print(pred.detach().numpy())
    #X_train['layer 11 predictions'] = pd.DataFrame(pred.detach().numpy())
    print(f'best loss: {best_loss} in epoch {best_epoch}\n')


Layer 11
best loss: 3.2517004013061523 in epoch 1000

Layer 10


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1540x66 and 67x45)